In [1]:

import os
import torch
from torch.utils.data import Dataset, DataLoader
import pydicom
import numpy as np
from torchvision import transforms
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch.nn as nn
from torch.utils.data import random_split
from model import MedicalImageCNN, CNNToRNA,CNNClassifier, train_model
import matplotlib.pyplot as plt

from helper import collate_fn
device = torch.device("cpu")

this shows that the image loader works, in rna preprocessing we already sorted everything so no worries and can take index to simply get the indexeth important gene

In [2]:

from helper import PatientDicomDatasetSingleGeneByIndexClass

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
if not os.path.exists('data/Images/NSCLC Radiogenomics'):
         raise FileNotFoundError("The directory 'data/Images/NSCLC Radiogenomics' does not exist.")
if not os.path.exists('data/df_zscore.csv'):
         raise FileNotFoundError("The file 'data/df_zscore.csv' does not exist.")
# Verify that the dataset is properly loaded
dataset =PatientDicomDatasetSingleGeneByIndexClass(root_dir='data/Images/NSCLC Radiogenomics', csv_path='data/df_zscore.csv', transform=transform, index=3)

# Check the length of the dataset to ensure it is not empty
if len(dataset) == 0:
    raise ValueError("The dataset is empty. Please check the root_dir and csv_path for correctness.")
print("Dataset length:", len(dataset))

images, patient_id, gene_value, gene_name = dataset[2]

print("Image tensor shape:", images[0].shape)
print('gene_value:', gene_value)
print('gene_name:', gene_name)

Loading CSV and images...
⚠️ Skipping patient R01-105 due to NaN gene value for NAA38
⚠️ Skipping patient R01-117 due to NaN gene value for NAA38
Dataset length: 128
Image tensor shape: torch.Size([1, 224, 224])
gene_value: tensor(2)
gene_name: NAA38


In [3]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

now we are going to train a model for each gene and extract the train loses and val losses to identify meaningful genes

lets do the top 10

In [4]:
def get_gene_index(gene_name):
    """
    Get the index of a gene by its name.
    :param gene_name: Name of the gene.
    :return: Index of the gene or None if not found.
    """
    try:
        return df_zscore[df_zscore['gene'] == gene_name].index[0]
    except IndexError:
        return None
    
df_zscore = pd.read_csv('data/df_zscore.csv')
gene_indexes = df_zscore['gene'].tolist()

In [ ]:
# Load the gene performance data from the saved file
gene_performance_path = 'gene_performance3.pth'
if not os.path.exists(gene_performance_path):
    raise FileNotFoundError(f"The file '{gene_performance_path}' does not exist.")
    

gene_performance = torch.load(gene_performance_path, weights_only=False)
print("Loaded gene performance data.")

# Reconstruct the gene_performance dictionary
gene_performance_df = pd.read_csv('tables/gene_performance_table_lt_0.6.csv')
gene_performance_thresholded = {}

for _, row in gene_performance_df.iterrows():
    gene_name = row['Gene']
    if gene_name in gene_performance:
        gene_performance_thresholded[gene_name] = gene_performance[gene_name]

print(f"Reconstructed gene_performance_thresholded with {len(gene_performance_thresholded)} entries.")


Loaded gene performance data.
Reconstructed gene_performance_thresholded with 38 entries.


saved all the relevant data so can go back to it in case it crashes

problem there is a massive class embalence

In [ ]:
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

def evaluate_model_auc(model, dataloader, device):
    model.eval()
    all_probs = []
    all_targets = []

    with torch.no_grad():
        for images, _, labels, _ in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)  # [B, num_classes]
            probs = F.softmax(outputs, dim=1)  # Convert logits to probabilities

            all_probs.append(probs.cpu())
            all_targets.append(labels.cpu())

    probs = torch.cat(all_probs).numpy()
    targets = torch.cat(all_targets).numpy()

    try:
        auc = roc_auc_score(targets, probs, multi_class='ovo', average='macro')  # 'ovr' also valid
    except ValueError:
        auc = float('nan')  # Handle degenerate cases

    return auc


data is super skewed

In [7]:
import random
from sklearn.model_selection import train_test_split
from helper import PatientDicomDatasetSingleGeneByIndexClassBinary, zscore_to_class_binary
from collections import Counter

#extract the list of indexes of the genes with less than 0.6 loss
gene_indexes = [get_gene_index(gene) for gene in gene_performance_thresholded.keys() if get_gene_index(gene) is not None]


criterion = nn.CrossEntropyLoss()  # Assuming a classification task, change to MSELoss for regression tasks

batch_size = 1
#want to store the models for each gene, the list of losses, list of vals, gene name as uid
gene_performance = {}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

for gene_idx in gene_indexes:
    

    # Update the dataset index for the current gene
    dataset = PatientDicomDatasetSingleGeneByIndexClassBinary(
        root_dir='data/Images/NSCLC Radiogenomics',
        csv_path='data/df_zscore.csv',
        transform=transform,
        index=gene_idx
    )
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    # Extract labels for stratification
    labels = [zscore_to_class_binary(float(val)) for val in dataset.gene_values]
    print(f"Labels for gene {gene_idx}: {labels}")

    label_counts = Counter(labels)
    if min(label_counts.values()) < 2:
        print(f"⚠️ Not enough samples to stratify for gene {gene_idx}. Doing random split.")
        indices = list(range(len(dataset)))
        random.shuffle(indices)
        train_size = int(0.7 * len(indices))
        val_size = int(0.15 * len(indices))
        train_indices = indices[:train_size]
        val_indices = indices[train_size:train_size+val_size]
        test_indices = indices[train_size+val_size:]
    else:
    # do stratified split as above

    # Perform stratified split

        train_indices, temp_indices = train_test_split(
            range(len(labels)), test_size=0.3, stratify=labels, random_state=42
        )
        val_indices, test_indices = train_test_split(
            temp_indices, test_size=0.5, stratify=[labels[i] for i in temp_indices], random_state=42
        )

    # Create subsets
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)
    _,_,_,gene_name = dataset[0]
    print(f"Training model for gene: {gene_name} (Index: {gene_idx})")
    # Update DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

    # Create a new model for the current gene
    model = CNNClassifier(
        cnn_encoder=MedicalImageCNN(output_dim=128),
        embedding_dim=128,
        num_classes=2  # Single output for the current gene
    )
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


    # Train the model
    train_losses, val_losses = train_model(
        model, train_loader, val_loader,
        criterion, optimizer, device, patience=2
    )

    # Store the losses for the current gene
    gene_performance[gene_name] = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'model': model,
        'total_lost': val_losses[-1],  # Store the last validation loss
        'gene_name': gene_name,
        'test_dataset': test_dataset,
        'test_accuracy':evaluate_model_auc(model, test_loader, device),
        'index': gene_idx,
        'dataset_length': len(dataset)
    }

    print(f"Finished training for gene: {gene_name}")
    torch.save(gene_performance, 'gene_performance.pth')
    model.to('cpu')
    torch.cuda.empty_cache()


#sort by total_lost in descending order
gene_performance = dict(sorted(gene_performance.items(), key=lambda item: item[1]['total_lost']))

# Visualize losses for top 10 genes
top_genes = list(gene_performance.keys())[:10]


Using device: cuda
Loading CSV and images...
Labels for gene 46: [1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1]
Training model for gene: VGLL4 (Index: 46)

Epoch [1/50]


Train Loss: 0.7311 | Val Loss: 0.6925

Epoch [2/50]


Train Loss: 0.6917 | Val Loss: 0.6921

Epoch [3/50]


Train Loss: 0.6908 | Val Loss: 0.6919
⏹️ Early stopping triggered at epoch 3
Finished training for gene: VGLL4
Loading CSV and images...
Labels for gene 67: [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
Training model for gene: USP32 (Index: 67)

Epoch [1/50]


Train Loss: 0.7169 | Val Loss: 0.6822

Epoch [2/50]


...

Train Loss: 0.4386 | Val Loss: 0.4560
⏹️ Early stopping triggered at epoch 3
Finished training for gene: CASP2
Loading CSV and images...
Labels for gene 62: [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Training model for gene: DNM1L (Index: 62)

Epoch [1/50]


Train Loss: 1.4823 | Val Loss: 0.5857

Epoch [2/50]


Train Loss: 0.5743 | Val Loss: 0.5391

Epoch [3/50]


Train Loss: 0.5401 | Val Loss: 0.5493

Epoch [4/50]


Train Loss: 0.4965 | Val Loss: 0.6033
⏹️ Early stopping triggered at epoch 4
Finished training for gene: DNM1L


In [1]:
def evaluate_model_auc(model, dataloader, device):
    model.eval()
    all_probs = []
    all_targets = []

    with torch.no_grad():
        for images, _, labels, _ in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)  # [B, num_classes]
            probs = F.softmax(outputs, dim=1)  # Convert logits to probabilities

            all_probs.append(probs.cpu())
            all_targets.append(labels.cpu())

    # Concatenate all probabilities and targets
    probs = torch.cat(all_probs).numpy()
    targets = torch.cat(all_targets).numpy()

    # Ensure targets are 1D
    if len(targets.shape) > 1:
        targets = np.argmax(targets, axis=1)  # Convert one-hot encoded labels to class indices

    if len(probs.shape) != 2:
        raise ValueError("Predicted probabilities must be a 2D array.")

    try:
        auc = roc_auc_score(targets, probs, multi_class='ovo', average='macro')  # 'ovr' also valid
    except ValueError as e:
        print(f"Error calculating AUC: {e}")
        auc = float('nan')  # Handle degenerate cases

    return auc


In [2]:
print("Top 10 genes:")

#load the gene performance data from the saved file
if not os.path.exists('gene_performance.pth'):
    raise FileNotFoundError("The file 'gene_performance.pth' does not exist.")
gene_performance = torch.load('gene_performance.pth', weights_only=False)

for gene in gene_performance.keys():
    #recalculate the test accuracy
    model = gene_performance[gene]['model']
    model.to(device)
    test_loader = DataLoader(gene_performance[gene]['test_dataset'], batch_size=1, shuffle=False, collate_fn=collate_fn)
    gene_performance[gene]['test_accuracy'] = evaluate_model_auc(gene_performance[gene]['model'], test_loader, device)
    print(gene, gene_performance[gene]['total_lost'], gene_performance[gene]['test_accuracy'])


Top 10 genes:


NameError: name 'os' is not defined